In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import pickle
import numpy as np
import pandas as pd
from joblib import load, Parallel, delayed
from tqdm import tqdm
from sklearn.covariance import LedoitWolf
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests

# -----------------------------
# Helper functions
# -----------------------------

def normalize_columns(df, col1='prot1', col2='prot2'):
    prot1_norm = np.where(df[col1] <= df[col2], df[col1], df[col2])
    prot2_norm = np.where(df[col1] <= df[col2], df[col2], df[col1])
    df[f'{col1}_norm'] = prot1_norm
    df[f'{col2}_norm'] = prot2_norm
    return df

def merge_preserve_left(left_df, right_df, col1='prot1', col2='prot2'):
    left_norm = normalize_columns(left_df.copy(), col1, col2)
    right_norm = normalize_columns(right_df.copy(), col1, col2)
    merged_df = pd.merge(
        left_norm,
        right_norm,
        left_on=[f'{col1}_norm', f'{col2}_norm'],
        right_on=[f'{col1}_norm', f'{col2}_norm'],
        how='left',
        suffixes=('_left', '_right')
    )
    merged_df.drop([f'{col1}_norm', f'{col2}_norm'], axis=1, inplace=True)
    if f'{col1}_right' in merged_df.columns: merged_df.drop(f'{col1}_right', axis=1, inplace=True)
    if f'{col2}_right' in merged_df.columns: merged_df.drop(f'{col2}_right', axis=1, inplace=True)
    if f'{col1}_left' in merged_df.columns: merged_df.rename(columns={f'{col1}_left': col1}, inplace=True)
    if f'{col2}_left' in merged_df.columns: merged_df.rename(columns={f'{col2}_left': col2}, inplace=True)

    # 处理其他重复列
    left_cols = set(left_df.columns)
    right_cols = set(right_df.columns)
    common_cols = left_cols.intersection(right_cols) - {col1, col2}
    for col in common_cols:
        left_col = f'{col}_left'
        right_col = f'{col}_right'
        if left_col in merged_df.columns and right_col in merged_df.columns:
            merged_df[col] = merged_df[left_col].combine_first(merged_df[right_col])
            merged_df.drop([left_col, right_col], axis=1, inplace=True)

    return merged_df

# -----------------------------
# Fast correlation computation
# -----------------------------
def fast_corr(expr_df, min_samples=30):
    genes = expr_df.index
    if expr_df.shape[1] < min_samples:
        raise ValueError("Too few samples to compute correlations")

    X = expr_df.T.values

    # Pearson correlation
    pearson_mat = expr_df.T.corr(min_periods=min_samples).fillna(0.0)

    # Partial correlation using LedoitWolf shrinkage
    try:
        lw = LedoitWolf().fit(X)
        prec = np.linalg.inv(lw.covariance_)
        D = np.diag(1 / np.sqrt(np.diag(prec)))
        pcorr_mat = - D.dot(prec).dot(D)
        np.fill_diagonal(pcorr_mat, 1.0)
        pcorr_mat = pd.DataFrame(pcorr_mat, index=genes, columns=genes)
    except Exception as e:
        print(f"LedoitWolf error: {e}; filling pcorr with zeros")
        pcorr_mat = pd.DataFrame(0.0, index=genes, columns=genes)

    # 上三角展开
    iu = np.triu_indices_from(pearson_mat, k=1)
    idx = pd.MultiIndex.from_arrays([pearson_mat.index[iu[0]], pearson_mat.columns[iu[1]]], names=['prot1','prot2'])
    pearson_series = pd.Series(pearson_mat.values[iu], index=idx, name='pearson')

    iu2 = np.triu_indices_from(pcorr_mat, k=1)
    idx2 = pd.MultiIndex.from_arrays([pcorr_mat.index[iu2[0]], pcorr_mat.columns[iu2[1]]], names=['prot1','prot2'])
    pcorr_series = pd.Series(pcorr_mat.values[iu2], index=idx2, name='pcorr')

    df = pd.concat([pearson_series, pcorr_series], axis=1).reset_index()
    df.columns = ['prot1','prot2','pearson','pcorr']
    return df

# -----------------------------
# Main permutation function
# -----------------------------
def run_permutation_with_pretrained(expr_normal, expr_cancer, go_df,
                                    model,
                                    n_perm=100, min_samples=30,
                                    pval_method="zscore",
                                    n_jobs=1):
    print("Computing observed correlations for normal ...")
    edge_normal = fast_corr(expr_normal, min_samples=min_samples)
    print("Computing observed correlations for cancer ...")
    edge_cancer = fast_corr(expr_cancer, min_samples=min_samples)

    edge_normal = merge_preserve_left(edge_normal, go_df)
    edge_cancer = merge_preserve_left(edge_cancer, go_df)

    feature_cols = ['pearson','pcorr','GO:MF','GO:BP','GO:CC']
    for c in feature_cols:
        if c not in edge_normal.columns: edge_normal[c]=0.0
        if c not in edge_cancer.columns: edge_cancer[c]=0.0

    all_edges = edge_normal.set_index(['prot1','prot2']).index.union(
        edge_cancer.set_index(['prot1','prot2']).index
    )
    all_edges = pd.MultiIndex.from_tuples(sorted(list(all_edges)))

    def build_feat(edge_idxed):
        df = pd.DataFrame(index=all_edges)
        for col in feature_cols:
            df[col] = edge_idxed.set_index(['prot1','prot2']).reindex(all_edges)[col].fillna(0.0).values
        return df

    feat_norm_all = build_feat(edge_normal)
    feat_canc_all = build_feat(edge_cancer)

    prob_normal = model.predict_proba(feat_norm_all.values)[:,1]
    prob_cancer = model.predict_proba(feat_canc_all.values)[:,1]

    result_df = pd.DataFrame(index=all_edges)
    result_df['prob_normal'] = prob_normal
    result_df['prob_cancer'] = prob_cancer
    result_df['diff_obs'] = result_df['prob_cancer'] - result_df['prob_normal']

    # ---------------------------
    # Permutation
    # ---------------------------
    combined_expr = pd.concat([expr_normal, expr_cancer], axis=1)
    sample_labels = np.array([0]*expr_normal.shape[1] + [1]*expr_cancer.shape[1])

    def permute_once(i):
        permuted_labels = np.random.permutation(sample_labels)
        group_a = combined_expr.loc[:, permuted_labels == 0]
        group_b = combined_expr.loc[:, permuted_labels == 1]

        edge_a = fast_corr(group_a, min_samples=min_samples)
        edge_b = fast_corr(group_b, min_samples=min_samples)

        edge_a = merge_preserve_left(edge_a, go_df)
        edge_b = merge_preserve_left(edge_b, go_df)

        feat_a = build_feat(edge_a)
        feat_b = build_feat(edge_b)

        prob_a = model.predict_proba(feat_a.values)[:,1]
        prob_b = model.predict_proba(feat_b.values)[:,1]

        return prob_b - prob_a

    perm_array = np.array(
        Parallel(n_jobs=n_jobs)(delayed(permute_once)(i) for i in tqdm(range(n_perm), desc="Permutations"))
    ).T  # shape = (n_edges, n_perm)

    # ---------------------------
    # P-value calculation
    # ---------------------------
    if pval_method == "empirical":
        abs_perm = np.abs(perm_array)
        abs_obs = np.abs(result_df['diff_obs'].values.reshape(-1,1))
        p_values = (np.sum(abs_perm >= abs_obs, axis=1) + 1) / (n_perm + 1)
    elif pval_method == "zscore":
        null_mean = np.mean(perm_array, axis=1)
        null_std = np.std(perm_array, axis=1, ddof=1)
        z_scores = (result_df['diff_obs'].values - null_mean) / null_std
        p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))
    else:
        raise ValueError("pval_method must be 'empirical' or 'zscore'")

    result_df['p_value'] = p_values
    _, fdr, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')
    result_df['fdr'] = fdr

    return result_df.reset_index()

# -----------------------------
# Main pipeline
# -----------------------------
def main_pipeline(expr_normal_file, expr_cancer_file,
                  go_path, model_path, 
                  output_dir='results_perm', n_perm=100,
                  min_samples=30, pval_method="zscore", n_jobs=1):

    os.makedirs(output_dir, exist_ok=True)

    print("Loading expression data ...")
    expr_normal = pd.read_csv(expr_normal_file, index_col=0)
    expr_cancer = pd.read_csv(expr_cancer_file, index_col=0)

    print("Loading GO features ...")
    go_df = pd.read_csv(go_path)

    print("Loading pretrained models ...")
    model = load(model_path)

    res = run_permutation_with_pretrained(expr_normal, expr_cancer, go_df,
                                          model,
                                          n_perm=n_perm,
                                          min_samples=min_samples,
                                          pval_method=pval_method,
                                          n_jobs=n_jobs)

    out_file = os.path.join(output_dir, "result_all_edges.csv")
    res.to_csv(out_file, index=False)
    print("Results saved to:", out_file)
    return res

# -----------------------------
# Script entry
# -----------------------------
if __name__ == "__main__":
    expr_normal_file = '0.processed data/LSCC_control_proteomics_processed.csv'
    expr_cancer_file = '0.processed data/LSCC_tumor_proteomics_processed.csv'
    go_path = '0.Data/Go_similarity.csv'
    model_path = '3.xgb_probabilities/LSCC_combined_proteomics_processed.csv_corr.obj_model.pkl'
    outdir = 'results_perm'

    res = main_pipeline(expr_normal_file, expr_cancer_file,
                        go_path, model_path,
                        output_dir=outdir,
                        n_perm=500,
                        min_samples=30,
                        pval_method="zscore",
                        n_jobs=8)


Loading expression data ...
Loading GO features ...
Loading pretrained models ...
Computing observed correlations for normal ...
Computing observed correlations for cancer ...


Permutations: 100%|████████████████████████████████████████████████████████████████| 500/500 [3:11:28<00:00, 22.98s/it]


Results saved to: results_perm\result_all_edges.csv


In [2]:
outdir = 'results_perm'
out_file = os.path.join(outdir, "result_all_edges.csv")
res.to_csv(out_file, index=False)

In [3]:
import xgboost as xgb
xgb.__version__

'3.0.4'

In [5]:
import sklearn as sk

In [9]:
sk.__version__

'1.5.1'